In [6]:
import kfp 
import kfp.dsl as dsl
from kfp.compiler import Compiler
from kfp.components import func_to_container_op

In [7]:
def add(a: float, b: float) -> float:
    print("Adding two values %s and %s" %(a, b))
    return a + b

def say(what: str):
    print(what)

    
    
add_op = func_to_container_op(func = add, base_image = "python:alpine")

say_op = func_to_container_op(func = say, base_image = "python:alpine")

In [8]:
@kfp.dsl.pipeline(
  name='CreatingLightWeightComponents',
  description='Creating lightweight components.'
)


def creating_lightweight_components(a='1', b='2', c='3', d='4',
    what='Completed!',
):
    add_task_1 = add_op(a, b) 
    add_task_2 = add_op(b, c)
    add_task_3 = add_op(c, d)
    add_task_4 = add_op(add_task_2.output, add_task_3.output)
    say_task   = say_op(what)

    add_task_2.after(add_task_1)
    add_task_3.after(add_task_1)
    add_task_4.after(add_task_2, add_task_3)
    say_task.after(add_task_4)

In [9]:
kfp.compiler.Compiler().compile(creating_lightweight_components,
                                'creating_lightweight_components.yaml')